In [11]:
import numpy as np

from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Estimator
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp

In [5]:
num_layers = 4
n_qubits = 3

inputs = ParameterVector("input", n_qubits)
weights = ParameterVector("weights", num_layers * n_qubits)

qc = QuantumCircuit(3)
for i in range(n_qubits):
    qc.rx(inputs[i], i)

for i in range(n_qubits):
    qc.ry(weights[i], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rz(weights[i+3], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rx(weights[i+6], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rz(weights[i+9], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

In [6]:
qc.draw()

┌──────────────┐┌────────────────┐          ┌───┐┌────────────────┐     »
q_0: ┤ Rx(input[0]) ├┤ Ry(weights[0]) ├──■───────┤ X ├┤ Rz(weights[3]) ├──■──»
     ├──────────────┤├────────────────┤┌─┴─┐     └─┬─┘├────────────────┤┌─┴─┐»
q_1: ┤ Rx(input[1]) ├┤ Ry(weights[1]) ├┤ X ├──■────┼──┤ Rz(weights[4]) ├┤ X ├»
     ├──────────────┤├────────────────┤└───┘┌─┴─┐  │  ├────────────────┤└───┘»
q_2: ┤ Rx(input[2]) ├┤ Ry(weights[2]) ├─────┤ X ├──■──┤ Rz(weights[5]) ├─────»
     └──────────────┘└────────────────┘     └───┘     └────────────────┘     »
«          ┌───┐┌────────────────┐          ┌───┐ ┌────────────────┐          »
«q_0: ─────┤ X ├┤ Rx(weights[6]) ├──■───────┤ X ├─┤ Rz(weights[9]) ├──■───────»
«          └─┬─┘├────────────────┤┌─┴─┐     └─┬─┘┌┴────────────────┤┌─┴─┐     »
«q_1: ──■────┼──┤ Rx(weights[7]) ├┤ X ├──■────┼──┤ Rz(weights[10]) ├┤ X ├──■──»
«     ┌─┴─┐  │  ├────────────────┤└───┘┌─┴─┐  │  ├─────────────────┤└───┘┌─┴─┐»
«q_2: ┤ X ├──■──┤ Rx(weights[8]) ├─────┤ X ├──■──┤ Rz(weights[11]) ├─────┤ X ├»
«     └───┘     └────────────────┘     └───┘     └─────────────────┘     └───┘»
«     ┌───┐
«q_0: ┤ X ├
«     └─┬─┘
«q_1: ──┼──
«       │  
«q_2: ──■──
«

In [8]:
obs = SparsePauliOp("ZZI")

In [12]:
service = QiskitRuntimeService()
backend = service.get_backend("ibmq_qasm_simulator")

In [13]:
weights_init = np.random.randn(num_layers * n_qubits)
inputs_init = np.random.randn(n_qubits)

In [18]:
qc.assign_parameters({weights:weights_init, inputs:inputs_init}, inplace=True)

In [19]:
qc.draw()

┌──────────────┐┌──────────────┐          ┌───┐┌─────────────┐           »
q_0: ─┤ Rx(-0.86752) ├┤ Ry(-0.45944) ├──■───────┤ X ├┤ Rz(-2.3229) ├───■───────»
      ├──────────────┤├──────────────┤┌─┴─┐     └─┬─┘├─────────────┴┐┌─┴─┐     »
q_1: ─┤ Rx(-0.20643) ├┤ Ry(-0.53726) ├┤ X ├──■────┼──┤ Rz(-0.88395) ├┤ X ├──■──»
     ┌┴──────────────┤└┬────────────┬┘└───┘┌─┴─┐  │  ├──────────────┤└───┘┌─┴─┐»
q_2: ┤ Rx(-0.010894) ├─┤ Ry(0.4082) ├──────┤ X ├──■──┤ Rz(-0.34143) ├─────┤ X ├»
     └───────────────┘ └────────────┘      └───┘     └──────────────┘     └───┘»
«     ┌───┐ ┌────────────┐           ┌───┐┌──────────────┐          ┌───┐
«q_0: ┤ X ├─┤ Rx(2.9705) ├───■───────┤ X ├┤ Rz(-0.30464) ├──■───────┤ X ├
«     └─┬─┘┌┴────────────┴┐┌─┴─┐     └─┬─┘├─────────────┬┘┌─┴─┐     └─┬─┘
«q_1: ──┼──┤ Rx(-0.26624) ├┤ X ├──■────┼──┤ Rz(0.14669) ├─┤ X ├──■────┼──
«       │  ├─────────────┬┘└───┘┌─┴─┐  │  ├─────────────┤ └───┘┌─┴─┐  │  
«q_2: ──■──┤ Rx(0.87922) ├──────┤ X ├──■──┤ Rz(0.32208) ├──────┤ X ├──■──
«          └─────────────┘      └───┘     └─────────────┘      └───┘

In [14]:
estimator = Estimator(backend)

In [20]:
job = estimator.run([qc], observables=[obs], shots=1024)

In [21]:
result = job.result()

In [22]:
result

EstimatorResult(values=array([0.12304688]), metadata=[{'variance': 0.9848594665527344, 'shots': 1024}])

CircuitError: 'Cannot bind parameters (weights[9], weights[5], weights[3], weights[11], weights[7], input[0], input[1], weights[10], weights[8], weights[6], weights[2], weights[1], weights[4], input[2], weights[0]) not present in the circuit.'

In [24]:
inputs

ParameterVector(name=input, length=3)

In [26]:
num_layers = 4
n_qubits = 3

inputs = ParameterVector("input", n_qubits)
weights = ParameterVector("weights", num_layers * n_qubits)
ks = ParameterVector("k", n_qubits)

qc = QuantumCircuit(3)
for i in range(n_qubits):
    qc.rx(inputs[i]*ks[i], i)

for i in range(n_qubits):
    qc.ry(weights[i], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rz(weights[i+3], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rx(weights[i+6], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

for i in range(n_qubits):
    qc.rz(weights[i+9], i)

for i in range(n_qubits):
    qc.cx(i, (i+1)%n_qubits)

In [27]:
qc.draw()

┌───────────────────┐┌────────────────┐          ┌───┐┌────────────────┐»
q_0: ┤ Rx(input[0]*k[0]) ├┤ Ry(weights[0]) ├──■───────┤ X ├┤ Rz(weights[3]) ├»
     ├───────────────────┤├────────────────┤┌─┴─┐     └─┬─┘├────────────────┤»
q_1: ┤ Rx(input[1]*k[1]) ├┤ Ry(weights[1]) ├┤ X ├──■────┼──┤ Rz(weights[4]) ├»
     ├───────────────────┤├────────────────┤└───┘┌─┴─┐  │  ├────────────────┤»
q_2: ┤ Rx(input[2]*k[2]) ├┤ Ry(weights[2]) ├─────┤ X ├──■──┤ Rz(weights[5]) ├»
     └───────────────────┘└────────────────┘     └───┘     └────────────────┘»
«               ┌───┐┌────────────────┐          ┌───┐ ┌────────────────┐     »
«q_0: ──■───────┤ X ├┤ Rx(weights[6]) ├──■───────┤ X ├─┤ Rz(weights[9]) ├──■──»
«     ┌─┴─┐     └─┬─┘├────────────────┤┌─┴─┐     └─┬─┘┌┴────────────────┤┌─┴─┐»
«q_1: ┤ X ├──■────┼──┤ Rx(weights[7]) ├┤ X ├──■────┼──┤ Rz(weights[10]) ├┤ X ├»
«     └───┘┌─┴─┐  │  ├────────────────┤└───┘┌─┴─┐  │  ├─────────────────┤└───┘»
«q_2: ─────┤ X ├──■──┤ Rx(weights[8]) ├─────┤ X ├──■──┤ Rz(weights[11]) ├─────»
«          └───┘     └────────────────┘     └───┘     └─────────────────┘     »
«          ┌───┐
«q_0: ─────┤ X ├
«          └─┬─┘
«q_1: ──■────┼──
«     ┌─┴─┐  │  
«q_2: ┤ X ├──■──
«     └───┘

In [28]:
result

EstimatorResult(values=array([0.12304688]), metadata=[{'variance': 0.9848594665527344, 'shots': 1024}])

In [29]:
result.values

array([0.12304688])